In [ ]:
  ! pip install wandb

     |████████████████████████████████| 1.8MB 7.1MB/s 
     |████████████████████████████████| 174kB 36.4MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 133kB 37.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=aac6bd755c4812467e8d2268cbbe47e96979b522ee1d6e16b147d02416990f90
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=20b90cc7eee65610f61452254eaa001c02b4c1ca8b8f85bff7ff6ea9aa15d460
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import wandb

In [ ]:
import numpy as np
import pandas as pd 

import tensorflow as tf
from tensorflow import keras
from keras.layers import Attention, Concatenate, TimeDistributed
import math
import random
from keras import Input, regularizers, Model
from keras.layers import Input
from keras.layers import LSTM, SimpleRNN, GRU
from keras.layers import Dense, Embedding
from keras.utils.vis_utils import plot_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv ("/content/drive/MyDrive/DLassignment3/hi.translit.sampled.train.tsv", sep = '\t')
data.rename(columns = {'अं' : 'Hindi', 'an' : 'English'}, inplace = True)


In [ ]:
data

,Hindi,English,3
0,अंकगणित,ankganit,3
1,अंकल,uncle,4
2,अंकुर,ankur,4
3,अंकुरण,ankuran,3
4,अंकुरित,ankurit,3
...,...,...,...
44198,ह्वेनसांग,hiuentsang,1
44199,ह्वेनसांग,hsuantsang,1
44200,ह्वेनसांग,hyensang,1
44201,ह्वेनसांग,xuanzang,1


In [ ]:
# batch_size = 64  # Batch size for training.
# epochs = 10  # Number of epochs to train for.
# latent_dim = 256  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for j in range(len(data)):
  try:
    input_text=data.iloc[j,1]
    target_text=data.iloc[j,0]
    target_text = "\t" + target_text + "\n"
    if(len(input_text)>=1 and len(target_text)>=2):
      input_texts.append(input_text)
      target_texts.append(target_text)
    for char in input_text:
      if char not in input_characters:
        input_characters.add(char)
    for char in target_text:
      if char not in target_characters:
        target_characters.add(char)
  except: print("error")
input_characters.add(' ')
target_characters.add(' ')

error
error


In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 44201
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [ ]:
  tmp_x_train = []
  tmp_x_train_index = random.sample([i for i in range(0,np.array(input_texts).shape[0])],math.ceil(np.array(input_texts).shape[0]*0.7))
  tmp_x_val = []
  tmp_y_train = []
  tmp_y_val = []
  for i in tmp_x_train_index:
    tmp_x_train.append(input_texts[i])
    tmp_y_train.append(target_texts[i])
  for i in range(np.array(input_texts).shape[0]):
    if i not in tmp_x_train_index:
      tmp_x_val.append(input_texts[i])
      tmp_y_val.append(target_texts[i])
  # for val in tmp_x_train[0]:
  #   print(input_characters[np.argmax(np.array(val))],end="")
  # print()
  # for val in tmp_y_train[0]:
  #   print(target_characters[np.argmax(np.array(val))],end="")
  # print()
  # for val in tmp_x_val[0]:
  #   print(input_characters[np.argmax(val)],end="")
  # print()
  # for val in tmp_y_val[0]:
  #   print(target_characters[np.argmax(val)],end="")
  # print()
  lim=int(len(tmp_x_train)*0.9)
  x_train = tmp_x_train[0:lim]
  y_train = tmp_y_train[0:lim]
  x_val = tmp_x_train[lim:]
  y_val = tmp_y_train[lim:]
  x_test = tmp_x_val
  y_test = tmp_y_val

  # x_train = np.array(sample_in[:math.ceil(np.array(sample_in).shape[0]*0.7)])
  # y_train = np.array(sample_out[:math.ceil(np.array(sample_out).shape[0]*0.7)])
  # x_val = np.array(sample_in[math.ceil(np.array(sample_in).shape[0]*0.7):])
  # y_val = np.array(sample_out[math.ceil(np.array(sample_out).shape[0]*0.7):])
  # x_train = np.array(sample_in)
  # y_train = np.array(sample_out)

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
m=max(max_encoder_seq_length, max_decoder_seq_length)

In [ ]:


encoder_input_data = np.zeros(
    (len(x_train), m, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(x_train), m, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(x_train), m, num_decoder_tokens), dtype="float32"
)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(x_train, y_train)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
encoder_input_val = np.zeros(
    (len(x_val), m, num_encoder_tokens), dtype="float32"
)
decoder_input_val = np.zeros(
    (len(x_val), m, num_decoder_tokens), dtype="float32"
)
decoder_target_val = np.zeros(
    (len(x_val), m, num_decoder_tokens), dtype="float32"
)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(x_val, y_val)):
    for t, char in enumerate(input_text):
        encoder_input_val[i, t, input_token_index[char]] = 1.0
    encoder_input_val[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_val[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_val[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_val[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_val[i, t:, target_token_index[" "]] = 1.0

In [ ]:
encoder_input_test = np.zeros(
    (len(x_test), m, num_encoder_tokens), dtype="float32"
)
decoder_input_test = np.zeros(
    (len(x_test), m, num_decoder_tokens), dtype="float32"
)
decoder_target_test = np.zeros(
    (len(x_test), m, num_decoder_tokens), dtype="float32"
)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(x_test, y_test)):
    for t, char in enumerate(input_text):
        encoder_input_test[i, t, input_token_index[char]] = 1.0
    encoder_input_test[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_test[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_test[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_test[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_test[i, t:, target_token_index[" "]] = 1.0

In [ ]:
encoder_input_test.shape

(13260, 21, 27)

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            # 'values': [1,2,3]
            # 'values': [5,7,9, 10,11]
            'values':[50,60,70,80,90,100]
        },
        'weight_decay':{
           #'values': [0, 1e-1, 1e-2, 1e-3,0.00099,0.00111]
            'values':[0]
          #  'values':[0.00099]
        },
         'batchSize':{
            'values':[64]
            # 'values':[8]
            # 'values':[16]
        },
        'regular_dropout':{
            #'values':[0.2,0.3,0.4]
             'values':[0]
            # 'values':[0.4]
        },
        'recurrent_dropout':{
            #'values':[0.2,0.3,0.4]
             'values':[0]
            # 'values':[0.2]
        },
        'regular_activation':{
            #'values': ['sigmoid','tanh','relu']
             'values':['sigmoid']
            # 'values':['relu']
        },
        'recurrent_activation':{
            #'values': ['sigmoid','tanh','relu']
            # 'values':['relu']
             'values':['sigmoid']
        },
        'no_of_hidden_layers':{
            'values':[1]
            # 'values':[2]
            # 'values':[3]
        },
        'hidden_layers_size':{
            'values':[64,128,256]
            # 'values':[256]
            # 'values':[128]
            # 'values':[16]
        },
        'cell_type':{
            #'values':['RNN','GRU','LSTM']
            'values':['RNN']
            # 'values':['GRU']
        }
        # 'train_word_accuracy':{
        #     'values':[0]
        # }
        # 'input_embedding_size':{
        #     'values':[16,32,64,256]
        # } 
        
        #'activation': {
            # 'values': ['relu', 'elu', 'selu', 'softmax']
         #   'values': ['sigmoid','tanh','ReLu']
        #}
        
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="dl_45_47", project="Attention_model")
#d7f2d3b3afd25dff68841ada514c88e687894723

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 342sej37
Sweep URL: https://wandb.ai/dl_45_47/Attention_model/sweeps/342sej37


In [ ]:
# target_characters

In [ ]:
# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])
# # print("Number of samples:", len(input_texts)
# print("Number of unique input tokens:", num_encoder_tokens)
# print("Number of unique output tokens:", num_decoder_tokens)
# print("Max sequence length for inputs:", max_encoder_seq_length)
# print("Max sequence length for outputs:", max_decoder_seq_length)

In [ ]:
# target_texts[0]

In [ ]:
# m=max(max_encoder_seq_length, max_decoder_seq_length)

In [ ]:
# ############## MADE A CHANGE AT MAX
# input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


In [ ]:

# encoder_input_data = np.zeros(
#     (len(input_texts), m, num_encoder_tokens), dtype="float32"
# )
# decoder_input_data = np.zeros(
#     (len(input_texts), m, num_decoder_tokens), dtype="float32"
# )
# decoder_target_data = np.zeros(
#     (len(input_texts), m, num_decoder_tokens), dtype="float32"
# )

In [ ]:
# for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
#     for t, char in enumerate(input_text):
#         encoder_input_data[i, t, input_token_index[char]] = 1.0
#     encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
#     for t, char in enumerate(target_text):
#         # decoder_target_data is ahead of decoder_input_data by one timestep
#         decoder_input_data[i, t, target_token_index[char]] = 1.0
#         if t > 0:
#             # decoder_target_data will be ahead by one timestep
#             # and will not include the start character.
#             decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
#     decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
#     decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
def RNN_model(config):
  # Define an input sequence and process it.
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.SimpleRNN(config.hidden_layers_size, return_sequences=True, return_state=True)
  encoder_outputs, state_1 = encoder(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_1]

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_lstm = keras.layers.SimpleRNN(config.hidden_layers_size, return_sequences=True, return_state=True)
  decoder_outputs, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  # print("decoder_outputs : ",decoder_outputs.shape)

  # Attention layer
  attn_layer = Attention(name='attention_layer')
  attn_out = attn_layer([decoder_outputs,encoder_outputs])
  # print("attn_out : ",attn_out.shape)

  # Concat attention input and decoder GRU output
  decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  # dense_time = TimeDistributed(decoder_dense, name='time_distributed_layer')
  decoder_outputs = decoder_dense(decoder_concat_input)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  encoder_model = keras.Model(encoder_inputs, [encoder_outputs, encoder_states])

  decoder_states_inputs = keras.Input(shape=(config.hidden_layers_size,))
  encoder_inf_output = keras.Input(shape=(None, config.hidden_layers_size))
  decoder_states_inputs = [decoder_states_inputs]

  decoder_outputs, state_1 = (keras.layers.SimpleRNN(config.hidden_layers_size, return_sequences=True, return_state=True))(decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_1]

  attn_layer = Attention(name='attention_layer')
  attn_inf_out = attn_layer([decoder_outputs, encoder_inf_output])
  decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs, attn_inf_out])
  # print(encoder_inf_output.shape)
  # print(decoder_outputs.shape)
  # print(attn_inf_out.shape)

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_inf_concat)
  decoder_model = keras.Model(
      [decoder_inputs] + [encoder_inf_output] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )  

  return model, encoder_model, decoder_model

In [ ]:
def LSTM_model(config):
  # Define an input sequence and process it.
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.LSTM(config.hidden_layers_size, return_sequences=True, return_state=True)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_h, state_c]

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_lstm = keras.layers.LSTM(config.hidden_layers_size, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  # print("decoder_outputs : ",decoder_outputs.shape)

  # Attention layer
  attn_layer = Attention(name='attention_layer')
  attn_out = attn_layer([decoder_outputs,encoder_outputs])
  # print("attn_out : ",attn_out.shape)

  # Concat attention input and decoder GRU output
  decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  # dense_time = TimeDistributed(decoder_dense, name='time_distributed_layer')
  decoder_outputs = decoder_dense(decoder_concat_input)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  encoder_model = keras.Model(encoder_inputs, [encoder_outputs, encoder_states])

  decoder_state_input_h = keras.Input(shape=(config.hidden_layers_size,))
  decoder_state_input_c = keras.Input(shape=(config.hidden_layers_size,))
  encoder_inf_output = keras.Input(shape=(None, config.hidden_layers_size))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

  decoder_outputs, state_h_dec, state_c_dec = (keras.layers.LSTM(config.hidden_layers_size, return_sequences=True, return_state=True))(decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_h_dec, state_c_dec]

  attn_layer = Attention(name='attention_layer')
  attn_inf_out = attn_layer([decoder_outputs, encoder_inf_output])
  decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs, attn_inf_out])
  # print(encoder_inf_output.shape)
  # print(decoder_outputs.shape)
  # print(attn_inf_out.shape)

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_inf_concat)
  decoder_model = keras.Model(
      [decoder_inputs] + [encoder_inf_output] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )  

  return model, encoder_model, decoder_model

In [ ]:
def GRU_model(config):
  # Define an input sequence and process it.
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.GRU(config.hidden_layers_size, return_sequences=True, return_state=True)
  encoder_outputs, state_1 = encoder(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_1]

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_lstm = keras.layers.GRU(config.hidden_layers_size, return_sequences=True, return_state=True)
  decoder_outputs, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  # print("decoder_outputs : ",decoder_outputs.shape)

  # Attention layer
  attn_layer = Attention(name='attention_layer')
  attn_out = attn_layer([decoder_outputs,encoder_outputs])
  # print("attn_out : ",attn_out.shape)

  # Concat attention input and decoder GRU output
  decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  # dense_time = TimeDistributed(decoder_dense, name='time_distributed_layer')
  decoder_outputs = decoder_dense(decoder_concat_input)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  encoder_model = keras.Model(encoder_inputs, [encoder_outputs, encoder_states])

  decoder_states_inputs = keras.Input(shape=(config.hidden_layers_size,))
  encoder_inf_output = keras.Input(shape=(None, config.hidden_layers_size))
  decoder_states_inputs = [decoder_states_inputs]

  decoder_outputs, state_1 = (keras.layers.GRU(config.hidden_layers_size, return_sequences=True, return_state=True))(decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_1]

  attn_layer = Attention(name='attention_layer')
  attn_inf_out = attn_layer([decoder_outputs, encoder_inf_output])
  decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs, attn_inf_out])
  # print(encoder_inf_output.shape)
  # print(decoder_outputs.shape)
  # print(attn_inf_out.shape)

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_inf_concat)
  decoder_model = keras.Model(
      [decoder_inputs] + [encoder_inf_output] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )  

  return model, encoder_model, decoder_model

In [ ]:
def decode_sequence(input_seq,encoder_model, decoder_model):
    # Encode the input as state vectors.
    encoder_outputs, states_value = encoder_model.predict(input_seq)
    # enc_latent_dim = np.array(encoder_outputs).shape
    # print(np.array(encoder_outputs).shape, np.array(states_value).shape)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    # cnt = 0
    while not stop_condition:
        returned = decoder_model.predict([target_seq] + [encoder_outputs] + states_value)
        output_tokens = returned[0]
        states_value = returned[1:]
        # cnt += 1
        # if(cnt == 20):
        #   break

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        # states_value = [h, c]
    return decoded_sentence

In [ ]:
def accuracy_train( encoder_model, decoder_model):
  count_test=0
  count_train=0
  num_sam=100
  for i in range(num_sam):
    seq_index=random.randint(0,len(x_test))
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_test[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,encoder_model, decoder_model)
    if(decoded_sentence[0:-1]==y_test[seq_index][1:-1]):
      count_test=count_test+1
    print("test_Input word:", x_test[seq_index])
    print("test_expected word:", y_test[seq_index][1:-1])
    print("test_predicted word:", decoded_sentence[0:-1])
    print(" ")
  for i in range(num_sam):
    seq_index=random.randint(0,len(x_train))
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,encoder_model, decoder_model)
    if(decoded_sentence[0:-1]==y_train[seq_index][1:-1]):
      count_train=count_train+1
    print("train_Input word:", x_train[seq_index])
    print("train_expected word:", y_train[seq_index][1:-1])
    print("train_predicted word:", decoded_sentence[0:-1])
    print(" ")
  return count_test/num_sam,count_train/num_sam

In [ ]:
# The sweep calls this function with each set of hyperparameters
def train():
  # Default values for hyper-parameters we're going to sweep over
  #

    config_defaults = {
        'epochs': 5,
        'weight_decay': 1e-2,
         'batchSize':16,
        'regular_dropout':0.2,
        'recurrent_dropout':0.2,
        'no_of_hidden_layers':1,
        'hidden_layers_size':256,
        'cell_type':'LSTM'
    }

    # SUFFLE TRAIN AND VAL INPUT
    # tmp_x_train_index = random.sample([i for i in range(np.array(sample_in).shape[0])],math.ceil(np.array(sample_in).shape[0]*0.7))
    # for i in tmp_x_train_index:
    #   tmp_x_train.append(sample_in[i])
    #   tmp_y_train.append(sample_out[i])
    # for i in range(np.array(sample_in).shape[0]):
    #   if i not in tmp_x_train_index:
    #     tmp_x_val.append(sample_in[i])
    #     tmp_y_val.append(sample_out[i])
    # for val in tmp_x_train[0]:
    #   print(alphabets[np.argmax(np.array(val))],end="")
    # print()
    # for val in tmp_y_train[0]:
    #   print(characters[np.argmax(np.array(val))],end="")
    # print()
    # for val in tmp_x_val[0]:
    #   print(alphabets[np.argmax(val)],end="")
    # print()
    # for val in tmp_y_val[0]:
    #   print(characters[np.argmax(val)],end="")
    # print()
    # x_train = np.array(tmp_x_train)
    # y_train = np.array(tmp_y_train)
    # x_val = np.array(tmp_x_val)
    # y_val = np.array(tmp_y_val)


    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    if config.cell_type == 'RNN':
      model, encoder_model, decoder_model = RNN_model(config)
    elif config.cell_type == 'LSTM':
      model, encoder_model, decoder_model = LSTM_model(config)
    elif config.cell_type == 'GRU':
      model, encoder_model, decoder_model = GRU_model(config)
    
    plot_model(model, to_file='model.png', show_shapes=True)
    plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)
    plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)



    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit([encoder_input_data, decoder_input_data],decoder_target_data, epochs=config.epochs,validation_data=([encoder_input_val, decoder_input_val],decoder_target_val), batch_size=config.batchSize)

    acc_test,acc_train=accuracy_train( encoder_model, decoder_model)    

    accuracy, loss, val_accuracy, val_loss = history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss']

    # wandb.log({'val_loss' : val_loss, 'val_accuracy': val_accuracy})

    for i in range(config.epochs):
      wandb.log({'val_loss' : val_loss[i], 'val_accuracy': val_accuracy[i],'loss' : loss[i], 'accuracy': accuracy[i], 'epoch': i})
    # wandb.finish()
    print("test word accuracy:",acc_test)
    print("train word accuracy:",acc_train)

    wandb.log({'test_accuracy': acc_test, 'train_accuracy': acc_train})
    
    

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
# num_encoder_tokens

In [ ]:
# # Define an input sequence and process it.
# encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
# encoder = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# # We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]

# # Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# # We set up our decoder to return full output sequences,
# # and to return internal states as well. We don't use the
# # return states in the training model, but we will use them in inference.
# decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# print("decoder_outputs : ",decoder_outputs.shape)

# # Attention layer
# attn_layer = Attention(name='attention_layer')
# attn_out = attn_layer([decoder_outputs,encoder_outputs])
# print("attn_out : ",attn_out.shape)

# # Concat attention input and decoder GRU output
# decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
# # dense_time = TimeDistributed(decoder_dense, name='time_distributed_layer')
# decoder_outputs = decoder_dense(decoder_concat_input)

# # Define the model that will turn
# # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
# model.compile(
#     optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
# )
# model.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.2,
# )

In [ ]:
  # encoder_model = keras.Model(encoder_inputs, [encoder_outputs, encoder_states])

In [ ]:
# plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)

In [ ]:
# decoder_state_input_h = keras.Input(shape=(latent_dim,))
# decoder_state_input_c = keras.Input(shape=(latent_dim,))
# encoder_inf_output = keras.Input(shape=(None, latent_dim))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_outputs, state_h_dec, state_c_dec = (keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True))(decoder_inputs, initial_state=decoder_states_inputs)
# decoder_states = [state_h_dec, state_c_dec]

# attn_layer = Attention(name='attention_layer')
# attn_inf_out = attn_layer([decoder_outputs, encoder_inf_output])
# decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs, attn_inf_out])
# print(encoder_inf_output.shape)
# print(decoder_outputs.shape)
# print(attn_inf_out.shape)

# decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
# decoder_outputs = decoder_dense(decoder_inf_concat)
# decoder_model = keras.Model(
#     [decoder_inputs] + [encoder_inf_output] + decoder_states_inputs, [decoder_outputs] + decoder_states
# )

In [ ]:
# plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)

In [ ]:
# reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
# reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
# def decode_sequence(input_seq):
#     # Encode the input as state vectors.
#     encoder_outputs, states_value = encoder_model.predict(input_seq)
#     # enc_latent_dim = np.array(encoder_outputs).shape
#     # print(np.array(encoder_outputs).shape, np.array(states_value).shape)
#     # Generate empty target sequence of length 1.
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     # Populate the first character of target sequence with the start character.
#     target_seq[0, 0, target_token_index["\t"]] = 1.0

#     # Sampling loop for a batch of sequences
#     # (to simplify, here we assume a batch of size 1).
#     stop_condition = False
#     decoded_sentence = ""
#     # cnt = 0
#     while not stop_condition:
#         output_tokens, h, c = decoder_model.predict([target_seq] + [encoder_outputs] + states_value)
#         # cnt += 1
#         # if(cnt == 20):
#         #   break

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_char = reverse_target_char_index[sampled_token_index]
#         decoded_sentence += sampled_char

#         # Exit condition: either hit max length
#         # or find stop character.
#         if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq = np.zeros((1, 1, num_decoder_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.0

#         # Update states
#         states_value = [h, c]
#     return decoded_sentence

In [ ]:
# count=0
# for seq_index in range(20):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index : seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     if(decoded_sentence[0:-1]==data.iloc[seq_index,0]):
#       count=count+1
    
#     print("-",count)
#     print("Input word:", input_texts[seq_index])
#     print("Input type:", len(input_texts[seq_index]))
#     print("expected word:", data.iloc[seq_index,0])
#     print("expected type:", len(data.iloc[seq_index,0]))
#     print("predicted word:", decoded_sentence)
#     print("predicted type:", len(decoded_sentence))
# print("word accuracy:",count/20)

In [ ]:
# count